In [ ]:
import re

def clean_product_name_final(product_name):
    """제품명에서 모든 부가 설명을 제거하고 핵심 제품명만 추출합니다."""
    name = product_name

    # 1. 괄호와 대괄호 안의 내용 전체 제거: 기획/증정/세트 정보 삭제
    name = re.sub(r'\[.*?\]', '', name).strip()
    name = re.sub(r'\(.*?\)', '', name).strip()

    # 2. 용량 및 수량 정보 제거: 100ml, 50g, 2ea 등 숫자+단위 제거
    # 숫자가 포함된 용량 단위 패턴을 제거합니다. (100ml, 50g 등)
    name = re.sub(r'\s*\d+[\s\d]*[mMgG][lLgG]|[\s\d]*[eE][aA]', '', name, flags=re.IGNORECASE).strip()

    # 3. 핵심 마케팅 키워드 제거
    # 자주 등장하는 마케팅/부가 설명 키워드를 제거합니다.
    keywords_to_remove = ['기획', '증정', '대용량', '더블', '세트', '단품', '올영픽', '리즈PICK', '수분광', '수분천재크림', '점보']
    for keyword in keywords_to_remove:
        # 단어 경계(\b)로 정확하게 키워드만 제거
        name = re.sub(r'\b' + re.escape(keyword) + r'\b', '', name, flags=re.IGNORECASE).strip()

    # 4. 연속된 공백을 하나로 줄이고 최종 정리
    name = re.sub(r'\s+', ' ', name).strip()
    
    return name if name else product_name

# 예시: 라로슈포제 시카플라스트 멀티 리페어 크림

1. 정리 후: 라로슈포제 시카플라스트 멀티 리페어 크림
2. 정리 후: 에스네이처 아쿠아 스쿠알란 수분크림


In [ ]:
def clean_and_split_ingredients(ingredients_list, api_ingredient_names):
    """
    성분 리스트를 분해하고 불순물을 제거하며, 공공 API 명칭을 기준으로 검증합니다.
    
    :param ingredients_list: 스크래핑된 원본 성분 리스트 (예: ["정제수 다이카프릴릴에터", "스쿠알란(150,000ppm)"])
    :param api_ingredient_names: 공공 API에서 추출한 표준 성분명 리스트 (Set 형태 권장)
    :return: 정제된 성분 리스트
    """
    
    # 공공 API에서 가져온 성분명 리스트를 Set으로 변환하여 빠른 검색에 사용
    api_set = set(api_ingredient_names)
    
    # 1. 단일 텍스트로 결합 및 불순물 제거 준비
    # 리스트를 하나의 문자열로 결합하고, 불필요한 공백이나 개행문자를 제거합니다.
    raw_text = ' '.join(ingredients_list).replace('\n', ' ').strip()
    
    # ppm 등 농도 정보와 괄호 내부의 내용 제거 (성분명 분리에 방해되는 요소)
    raw_text = re.sub(r'\([^)]*\)', ' ', raw_text).strip()
    # 대괄호 안의 내용도 제거
    raw_text = re.sub(r'\[.*?\]', ' ', raw_text).strip()
    
    # 제품명/용량과 성분명이 붙어있는 경우를 대비해 자주 보이는 구분자(숫자, 용량단위 등)를 공백으로 치환
    raw_text = re.sub(r'\d+[\s\d]*[mMgG][lLgG]|\d+[\s\d]*[eE][aA]', ' ', raw_text, flags=re.IGNORECASE)
    
    # 2. 성분명으로 텍스트 재분리 (핵심 로직)
    tokenized_ingredients = set()
    current_text = raw_text
    
    # API 성분명과 매칭하여 성분 추출: 긴 성분명이 먼저 매칭되도록 길이 순으로 정렬
    # '소듐하이알루로네이트'가 '소듐'보다 먼저 추출되어야 정확합니다.
    sorted_api_list = sorted(list(api_set), key=len, reverse=True)

    for api_name in sorted_api_list:
        # 단어 경계(\b)를 사용하여 정확하게 성분명과 일치하는지 확인
        if re.search(r'\b' + re.escape(api_name) + r'\b', current_text):
            tokenized_ingredients.add(api_name)
            # 매칭된 성분명을 공백으로 치환하여 중복 매칭과 잔여 텍스트 오류를 방지
            current_text = re.sub(r'\b' + re.escape(api_name) + r'\b', ' ', current_text)

    # 3. 최종 불순물 제거 및 리스트 정리
    cleaned_list = []
    for item in tokenized_ingredients:
        item = item.strip()
        
        # 숫자로만 이루어진 잔여물이나 너무 짧은 단어(3자 이하) 제거
        if re.fullmatch(r'^\d+[\s\d]*$', item) or len(item) <= 3:
            continue
        
        # 최종적으로 API 표준 목록에 포함된 성분만 인정하여 중복 검증
        if item in api_set:
            cleaned_list.append(item)
            
    # 최종 중복 제거 및 정렬
    return sorted(list(set(cleaned_list)))

In [ ]:
import pandas as pd
import numpy as np
import re # 정규 표현식 라이브러리

# --- 설정 (파일 이름은 실제 파일명에 맞게 수정하세요) ---
COOS_FILE = "coos_ingredient_database_final.csv"
OLIVE_YOUNG_FILE = "oliveyoung_products.csv" # ⚠️ 실제 올리브영 파일명으로 수정 필요
OUTPUT_FILE = "integrated_cosmetic_data.csv"

# ----------------------------------------------------------------------
# 0. 데이터 불러오기
# ----------------------------------------------------------------------
try:
    # COOS 성분 데이터 (스크래핑 결과)
    df_coos = pd.read_csv(COOS_FILE)
    print(f"COOS 데이터 로드 완료: {len(df_coos)}개 성분")
except FileNotFoundError:
    print(f"❌ 오류: COOS 파일 '{COOS_FILE}'을 찾을 수 없습니다. 파일명을 확인해 주세요.")
    exit()

try:
    # 올리브영 상품 데이터 (이전에 스크래핑했던 결과)
    # ⚠️ 이 파일은 '상품명', '가격', '성분_목록' (콤마로 구분된 성분 리스트) 컬럼이 있다고 가정합니다.
    df_oly = pd.read_csv(OLIVE_YOUNG_FILE)
    print(f"올리브영 데이터 로드 완료: {len(df_oly)}개 상품")
except FileNotFoundError:
    # 테스트를 위해 올리브영 데이터가 없을 경우 가상 데이터 생성
    print(f"⚠️ 올리브영 파일 '{OLIVE_YOUNG_FILE}'이 없어 가상 데이터를 생성합니다.")
    df_oly = pd.DataFrame({
        '상품ID': [101, 102, 103],
        '상품명': ['A사 수딩 크림', 'B사 톤업 선크림', 'C사 샴푸'],
        '브랜드': ['A사', 'B사', 'C사'],
        '가격': [25000, 32000, 18000],
        # COOS 데이터와 매칭 테스트를 위한 성분 목록
        '성분_목록': [
            "정제수, 글리세린, 히비스커스꽃추출물, 히스티딘, 토코페롤",
            "정제수, 징크옥사이드, 티타늄디옥사이드, 흰버드나무껍질추출물, 나이아신아마이드",
            "라우릴글루코사이드, 코카미도프로필베타인, 정제수"
        ]
    })


# ----------------------------------------------------------------------
# 1단계: COOS 데이터 클리닝 및 표준화
# ----------------------------------------------------------------------
print("\n[1단계] COOS 성분 데이터 정리 시작...")

# 1. 태그 목록 정리 (불필요한 노이즈 제거)
# 스크래핑 코드에서 이미 필터링했으나, 혹시 모를 잔여 노이즈를 정리합니다.
def clean_tags(tags):
    if not isinstance(tags, str):
        return ""
    
    # 콤마로 분리 후 불필요한 패턴 제거
    tags_list = [t.strip() for t in tags.split(',')]
    
    # 길이가 짧거나, 특정 코드가 포함된 태그를 다시 제거 (예: AI, KO, EN, 식, 가능, 불가)
    NOISE_PATTERNS = re.compile(r'^(AI|KO|EN|JP|EU|식|가능|불가|EWG)$', re.IGNORECASE)
    
    cleaned_tags = [
        t for t in tags_list 
        if t and len(t) > 2 and not NOISE_PATTERNS.match(t)
    ]
    return ', '.join(cleaned_tags)

df_coos['태그_목록_정리'] = df_coos['태그_목록'].apply(clean_tags)

# 2. 표준 성분명 컬럼 생성 (조인을 위한 키)
# 원료명, 영문명 모두 공백, 괄호 등을 제거하여 조인 시 오류를 줄일 수 있습니다.
def create_standard_name(name):
    if not isinstance(name, str):
        return None
    # 띄어쓰기, 괄호 제거 후 소문자 변환
    return re.sub(r'[\s\(\)\[\]]+', '', name).lower()

df_coos['표준성분명'] = df_coos['원료명'].apply(create_standard_name)

print("COOS 데이터 정리 및 표준 성분 키 생성 완료.")


# ----------------------------------------------------------------------
# 2단계: 올리브영 데이터 성분 목록 분리 (Explode)
# ----------------------------------------------------------------------
print("\n[2단계] 올리브영 성분 목록 분리 시작...")

# 1. '성분_목록' 컬럼을 콤마(,) 기준으로 리스트로 변환
df_oly['성분_리스트'] = df_oly['성분_목록'].str.split(r',(?![^\(]*\))') 
# (팁: 괄호 안의 콤마는 분리하지 않는 정규표현식, 복잡한 성분명 처리)

# 2. explode (데이터 폭발): 하나의 상품을 여러 성분 행으로 분리
df_oly_exploded = df_oly.explode('성분_리스트')

# 3. 분리된 성분명 클리닝 및 표준 성분 키 생성
df_oly_exploded['원료명'] = df_oly_exploded['성분_리스트'].str.strip()
df_oly_exploded['표준성분명'] = df_oly_exploded['원료명'].apply(create_standard_name)

# 불필요한 행(빈 성분명) 제거
df_oly_exploded.dropna(subset=['표준성분명'], inplace=True)

print(f"올리브영 데이터 폭발 완료. 총 {len(df_oly_exploded)}개의 성분 항목 생성.")


# ----------------------------------------------------------------------
# 3단계: 표준 성분 매핑 및 데이터 통합 (JOIN)
# ----------------------------------------------------------------------
print("\n[3단계] 두 데이터셋 통합(Merge) 시작...")

# COOS 데이터에서 필요한 컬럼만 선택 (중복 조인을 막기 위해)
coos_cols = ['표준성분명', '원료명', '영문명', 'CAS_No', '설명_요약', '태그_목록_정리', '상세_URL']
df_coos_clean = df_coos[coos_cols].rename(columns={'원료명': 'COOS_원료명'}).drop_duplicates(subset=['표준성분명'])


# 올리브영 상품 데이터와 COOS 성분 데이터를 '표준성분명'을 기준으로 Left Join
# Left Join을 사용하면, COOS 데이터에 없는 성분이라도 올리브영 상품 정보는 유지됩니다.
df_final = pd.merge(
    df_oly_exploded, 
    df_coos_clean, 
    on='표준성분명', 
    how='left', 
    suffixes=('_OLY', '_COOS')
)

# ----------------------------------------------------------------------
# 4. 최종 정리 및 저장
# ----------------------------------------------------------------------

# 최종 컬럼 순서 지정 및 정리
df_final = df_final.rename(columns={'원료명_OLY': '사용_원료명_OLY'})

final_columns = [
    '상품명', '브랜드', '가격', # 상품 정보
    '사용_원료명_OLY', 'COOS_원료명', # 매핑된 성분명
    '영문명', 'CAS_No', '설명_요약', '태그_목록_정리', # COOS 성분 속성
    '상세_URL' # COOS 링크
]

df_final = df_final[[col for col in final_columns if col in df_final.columns]]

# 매핑되지 않은 성분에 대한 처리 (COOS 정보가 없는 경우)
df_final['COOS_매칭여부'] = np.where(df_final['CAS_No'].isnull(), '미매칭', '매칭완료')

df_final.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

print("\n==================================================")
print(f"✅ 데이터 통합 및 표준화 완료!")
print(f"최종 통합 데이터셋 크기: {len(df_final)} 행")
print(f"결과 파일: {OUTPUT_FILE}")
print("==================================================")
